# Baseline for Author vision
- uses selected values as a distance measure
- probability of having seen a tweet is reduced by a half with each step in the reply hierachy
- probability of having seen a tweet is reduced by a quarter for each step away from the root
- probabiliy of having seen a tweet is increased for each path in the follower network to the tweet (forthcoming)

In [2]:
import pickle

filename = "data/vision_graph_data.pkl"
with open(filename, 'rb') as f:
    df_baseline = pickle.load(f)


# in order to allow the comparison, the filter from the other notebook needs to run and the predictions, too
%run author_vision.ipynb
conversation_ids = set(df.conversation_id.tolist())
df_baseline = df_baseline[df_baseline["conversation_id"].isin(conversation_ids)]


cuda gpu is available: True


2022-08-16 15:55:46.479369: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-16 15:55:46.479721: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-16 15:55:46.479950: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-16 15:55:46.480229: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-16 15:55:46.480460: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

chosen 30 conversations and gotten 26251 from twitter compared to 24497 from reddit
reddit:
0    21624
1     2873
Name: y, dtype: int64
twitter:
0    24184
1     2067
Name: y, dtype: int64
(21000, 47)
(21000,)
2.6.0
165/165 [==============================] - 0s 1ms/step - loss: 0.3137 - accuracy: 0.9223 - mae: 0.2328
the accuracy on the training set is 0.9223005175590515 and the mae is 0.2328256070613861
(19597, 47)
(19597,)
2.6.0
154/154 [==============================] - 0s 867us/step - loss: 0.3967 - accuracy: 0.8786 - mae: 0.2791
the accuracy on the training set is 0.8785714507102966 and the mae is 0.27907034754753113
        timedelta  root_distance_0  reply_distance_2  reply_distance_3  \
264298   0.000044                0               1.0               0.0   
405226   0.000002                0               1.0               0.0   

        reply_distance_4  root_distance_1  root_distance_2  root_distance_3  \
264298               0.0              0.0              1.0          

In [3]:
# sum all reply columns

reply_filter_col = [col for col in df_baseline if col.startswith('reply')]
root_distance_filter_col = [col for col in df_baseline if col.startswith('root')]
# reply_filter_col

In [4]:
reply_columns = df_baseline[reply_filter_col]
root_distance_columns= df_baseline[root_distance_filter_col]
# reply_columns

In [5]:
reply_cs = reply_columns.sum(axis=1)
root_distance_cs = root_distance_columns.sum(axis=1)

rcs_not_null = [i for i in reply_cs.tolist() if i != 0]

In [6]:
root_reply_combined = (root_distance_cs + reply_cs)
root_reply_combined = (root_reply_combined - root_reply_combined.min()) / (root_reply_combined.max() - root_reply_combined.min())
combined = [i for i in root_reply_combined.tolist() if i != 0]

In [7]:
#
df_baseline = df_baseline.assign(root_reply_combined=root_reply_combined)
# df_baseline

In [8]:
import numpy as np

df_data = df_baseline[["root_reply_combined", "conversation_id", "current", "platform"]]
# df_reshaped = pd.pivot_table(df_data,index=["conversation_id", "current"], columns=["root_reply_combined"],aggfunc = np.mean)
baseline_gpm = df_data.groupby(["platform", "conversation_id",  "current"]).mean()
baseline_gpm

root_reply_combined
platform conversation_id     current                                 
delab    1455538619804831744 72709458                        0.500000
         1457613499748831233 572575                          0.681818
         1463938900544692229 83448782                        0.730769
         1463970148050681857 60138487                        0.517241
         1464222274265960454 87771230                        0.500000
...                                                               ...
twitter  1552558905275858946 1552563036443099137             0.038462
                             1552563089635250176             0.035714
                             1552563118403993600             0.033333
                             1552563228034715649             0.031250
                             1552563434188902400             0.029412

[135767 rows x 1 columns]

In [9]:
baseline_gpm_conversation = baseline_gpm.groupby(by=["platform", "conversation_id"]).mean()
baseline_gpm_conversation

root_reply_combined
platform conversation_id                         
delab    1455538619804831744             0.500000
         1457613499748831233             0.681818
         1463938900544692229             0.730769
         1463970148050681857             0.517241
         1464222274265960454             0.500000
...                                           ...
twitter  1552396506501713925             0.228333
         1552551318866120704             0.514063
         1552558203963154434             0.228333
         1552558701432774659             0.502094
         1552558905275858946             0.101163

[4299 rows x 1 columns]

In [10]:
baseline_gpm = baseline_gpm_conversation.groupby(by=["platform"]).mean()
baseline_gpm

,root_reply_combined
platform,
delab,0.436427
reddit,0.436952
twitter,0.135928


In [11]:
cross_method_gpm = baseline_gpm_conversation.join(gpm_per_conversation, how='inner')
# cross_method_gpm
cross_method_gpm.corr()

,root_reply_combined,predictions
root_reply_combined,1.000000,-0.267511
predictions,-0.267511,1.000000
